In [1]:
from scipy.signal import spectrogram
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import math
from scipy.io.wavfile import write
import scipy.signal
import IPython.display as ipd
from pydub import AudioSegment
import random
from os.path import join
from os import listdir
sr=16000
duration=16
filename = r"voc.mp3"

In [2]:
whale_song, _ = librosa.load(filename, sr)
whale_song=whale_song[int(33*sr):int((33+duration)*sr)]
whale_song, _ = librosa.effects.trim(whale_song)

D:\Ext. App Files\Code\Py\DataScience\venv_conda\Lib\site-packages\librosa\core\audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


In [ ]:
amplitude = 2048; freq=2**10
t = np.linspace(0, duration, int(sr * duration))
# whale_song = amplitude * np.sin(2 * np.pi * freq * t)

In [ ]:
fig = px.line(whale_song)
fig.show()
ipd.Audio(whale_song, rate=sr)

In [ ]:
f, t, stft = scipy.signal.stft(whale_song, sr)
stft_real = np.real(stft)
stft_imag = np.imag(stft)

In [3]:
notes_freq = np.arange(1,10)
notes_freq = 16.352*pow(2,notes_freq)

def audio_to_mel(audio_data):
    fmin=16.352
    mel_tensor=[]
    for i, fmax in enumerate(notes_freq):
        hop_length=sr//2; n_mels=32
        n_fft=[2**14,2**13,2**12,2**11]
        n_fft=n_fft[min(i,len(n_fft)-1)]
        S = librosa.power_to_db(
               librosa.feature.melspectrogram(
                   audio_data, sr=sr, n_fft=n_fft, hop_length=hop_length,
                   n_mels=n_mels, fmin=fmin, fmax=fmax),
            ref=np.max)

        # librosa.display.specshow(S, sr=sr, hop_length=hop_length,
        #         fmin=fmin, fmax=fmax, x_axis='time', y_axis='mel');
        # plt.colorbar(format='%+2.0f dB');
        # plt.show()

        fmin=fmax
        mel_tensor.append(S)

    return np.array(mel_tensor)

In [ ]:
whale_song, _ = librosa.load(r"D:\Ext. App Files\Code\Py\DataScience\Ex3"
            r"\melspectrogram\src_data\cure\The_Top\03-Wailing_Wall.mp3", sr)
whale_song, _ = librosa.effects.trim(whale_song)
whale_song=whale_song[int(33*sr):int((33+duration)*sr-1)]
S=audio_to_mel(whale_song)
print(S.shape)

In [4]:
class SplitWavAudioMubin():
    def __init__(self, outfolder, infolder, filename):
        self.outfolder = outfolder
        self.infolder = infolder
        self.filename = filename

        self.audio, _ = librosa.load(join(self.infolder, self.filename), sr)
        self.audio, _ = librosa.effects.trim(self.audio)

    def get_duration(self):
        return len(self.audio)/sr

    def multiple_split(self, sec_interv):
        print(self.filename)
        track_samples=[]
        total_time=self.get_duration()//sec_interv*sec_interv
        for t in np.arange(sec_interv, total_time, sec_interv):
            audio=self.audio[int((t-sec_interv)*sr):int(t*sr-1)]
            track_samples.append(audio_to_mel(audio))
            # print(track_samples[-1].shape)
        for t in np.arange(3*sec_interv//2, total_time+sec_interv//2, sec_interv):
            audio=self.audio[int((t-sec_interv)*sr):int(t*sr-1)]
            track_samples.append(audio_to_mel(audio))
            # print(track_samples[-1].shape)

        with open(join(self.outfolder,self.filename.split('.')[0]), 'wb') as f:
            np.savez_compressed(f,np.array(track_samples))

In [ ]:
all_list = open('src_data\\all.list').readlines()
all_list = [ss.replace('\n','') for ss in all_list]
all_list = [[join('src_data',*ss.split('/')[:-1]), ss.split('/')[-1]+'.mp3'] for ss in all_list]
for infolder, file in all_list:
    split_wav = SplitWavAudioMubin('working_data', infolder, file)
    split_wav.multiple_split(sec_interv=duration)
    # except: print('****',file,'****')


In [5]:
dataset=[]
for f in listdir('working_data'):
    dataset+=list(np.load(join('working_data',f))['arr_0'])
random.shuffle(dataset)
print(len(dataset))

38842


In [ ]:
with open('generated_dataset.npz', 'wb') as f:
    np.savez_compressed(f,np.array(dataset))

In [8]:
split_wav = SplitWavAudioMubin('testing', 'testing', 'd.wav')
split_wav.multiple_split(sec_interv=duration)



d.wav


D:\Ext. App Files\Code\Py\DataScience\venv_conda\Lib\site-packages\librosa\filters.py:238: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn(
